# Lab 2: Graph Classification

### Load packages

In [3]:
# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import torch_geometric.datasets as datasets
import torch_geometric.data as data
import torch_geometric.transforms as transforms
from torch_geometric.utils.convert import to_networkx

Matplotlib is building the font cache; this may take a moment.


Load **ENZYMES dataset** provided by the TU Dortmund University that has collected a wide range of different graph classification datasets, known as the [**TUDatasets**](https://chrsmrrs.github.io/datasets/), which are also accessible via [`torch_geometric.datasets.TUDataset`](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.TUDataset) in PyTorch Geometric.

## Load and format the dataset

In [6]:
# TO BE COMPLETED 
from torch_geometric.datasets import TUDataset

dataset = TUDataset()
print(f'Dataset: {dataset} is loaded')

TypeError: TUDataset.__init__() missing 2 required positional arguments: 'root' and 'name'

Investigate the dataset

In [ ]:
# TO BE COMPLETED

Split the dataset into training and test sets

In [ ]:
# TO BE COMPLETED

# Define train/test split manually with native slicing [450:]
num_nodes = data.num_nodes
split_idx = int(0.8 * num_nodes)  # 80% of the nodes for training

# Create boolean masks
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

# 80% for training (from index 0 to `split_idx`)
train_mask[:split_idx] = True

# 20% for testing (from index `split_idx` to the end)
test_mask[split_idx:] = True

# Add train and test mask to data object
data.train_mask = train_mask
data.test_mask = test_mask

Set the mini-batching

In [ ]:
# TO BE COMPLETED

## Implementing a Graph Neural Network for graph classification task

Define the architecture of a four layers GNN (3 hidden layers GCN + 1 fully-connected layer, don't forget readout operation):

In [ ]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 64) 
        self.conv2 = GCNConv(64, 128) 
        self.conv3 = GCNConv(128, 64)
        self.conv4 = GCNConv(64, 32)
        self.conv5 = GCNConv(32, 16)
        self.conv6 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x, edge_index = x, data.edge_index

        x = self.conv1(x, edge_index) 
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.conv5(x, edge_index)
        x = F.relu(x)
        x = self.conv6(x, edge_index)

        return F.log_softmax(x, dim=)

model = GCN()
print(model)

Define your training and evaluation loops

In [8]:
# TO BE COMPLETED

# TO BE COMPLETED
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask]) #use the training split predifined through attribute train_mask
    loss.backward()
    optimizer.step()


model.eval()#on the test split pre-defined through attribute test_mask
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

NameError: name 'torch' is not defined

In [ ]:
model.eval()#on the test split pre-defined through attribute test_mask
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')